In [ ]:
import subprocess
import sys
import os

def install_package(package_name):
    try:
        # Check if the package is installed
        subprocess.check_call([sys.executable, "-m", "pip", "show", package_name])
        print(f"{package_name} is already installed.")
    except subprocess.CalledProcessError:
        # If package is not installed, install it
        print(f"Installing {package_name}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])
        print(f"{package_name} installation completed.")

def install_ngrok():
    # Check if ngrok is already downloaded and extracted
    if not os.path.exists('ngrok'):
        print("Downloading and installing ngrok...")
        subprocess.run(['wget', '-q', '-O', 'ngrok.zip', 'https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.zip'])
        subprocess.run(['unzip', '-o', 'ngrok.zip'])
        print("ngrok installed successfully.")
    else:
        print("ngrok is already installed.")

# Installing packages step-by-step
install_package("flask")
install_package("python-dotenv")
install_package("flask-cors")
install_package("pyngrok")

# Install ngrok separately
install_ngrok()

print("Done With All installations >>>>>>>>>>>>>>>>> Move Forward")

# Entering the APIs to be Sotred

In [ ]:
import os
import getpass  # Import the getpass module to hide input text

# Function to create/update .env file with ngrok API key
def set_ngrok_api_key():
    # Ask user for the ngrok API key without displaying the input
    ngrok_api_key = getpass.getpass("Please enter your ngrok API key: ")
    
    # Check if .env file exists
    env_file = '.env'
    
    # If file exists, append the key; if not, create a new file
    if os.path.exists(env_file):
        with open(env_file, 'a') as file:
            file.write(f"NGROK_AUTHTOKEN={ngrok_api_key}\n")
    else:
        with open(env_file, 'w') as file:
            file.write(f"NGROK_AUTHTOKEN={ngrok_api_key}\n")
    
    print(f"ngrok API key added to {env_file}")



# Call the functions to set the API key and credentials
set_ngrok_api_key()

# Implementing the Flask API GateWays

In [ ]:
import os
import threading
import time
from flask import Flask, request, jsonify
from flask_cors import CORS  # Import CORS to handle cross-origin requests
from pyngrok import ngrok  # Import ngrok
from dotenv import load_dotenv  # Import the dotenv package to load .env file

# Load environment variables from .env file
load_dotenv()

# Initialize Flask app
app = Flask(__name__)

# Enable CORS for all routes
CORS(app)

# Test endpoint to check if the server is running
@app.route('/test', methods=['POST'])
def test():
    return jsonify({"message": "Flask server is running!"})


import requests  # Import the requests library for making HTTP requests

@app.route('/process-url', methods=['POST'])
def process_url():
    try:
        # Get data from the request
        data = request.get_json()
        if not data or 'fileUrl' not in data or 'ngrokUrl' not in data:
            return jsonify({"error": "Missing required fields 'fileUrl' or 'ngrokUrl'"}), 400

        file_url = data['fileUrl']
        ngrok_url = data['ngrokUrl']

        # Log the received data
        print(f"Received fileUrl: {file_url}")
        print(f"Received ngrokUrl: {ngrok_url}")

        # Prepare the payload for the POST request
        payload = {
            "fileUrl": file_url,
            "ngrokUrl": ngrok_url
        }

        # Send the POST request to the external endpoint
        external_endpoint = f"{ngrok_url}/api/v2/post-image-details"  # Construct the URL
        response = requests.post(external_endpoint, json=payload)

        # Check if the response was successful
        if response.status_code == 200:
            print(f"POST request successful: {response.json()}")
            return jsonify({"message": "POST request sent successfully", "response": response.json()}), 200
        else:
            print(f"POST request failed with status {response.status_code}: {response.text}")
            return jsonify({"error": "POST request failed", "details": response.text}), response.status_code

    except Exception as e:
        print(f"Error processing request: {e}")
        return jsonify({"error": "An error occurred while processing the request"}), 500


# Function to run Flask server
def run_flask():
    app.run(host="0.0.0.0", port=5000)  # Running on port 5000

# Function to start ngrok and expose the Flask server
def run_ngrok():
    ngrok_auth_token = os.getenv("NGROK_AUTHTOKEN")
    if ngrok_auth_token:
        ngrok.set_auth_token(ngrok_auth_token)
    else:
        print("Ngrok authentication failed. Please check your API key.")
        return None

    # Open a ngrok tunnel to the Flask app on port 5000
    public_url = ngrok.connect(5000)
    print(f"Ngrok public URL: {public_url}")
    return public_url

# Start Flask server in a separate thread to keep the program running
flask_thread = threading.Thread(target=run_flask)
flask_thread.daemon = True  # Allow thread to terminate when the program exits
flask_thread.start()

# Start ngrok and print the URL where Flask is accessible
public_url = run_ngrok()

# Keep the program running to maintain the server
try:
    while True:
        time.sleep(1)  # Keeps the program running
except KeyboardInterrupt:
    print("Server shutting down.")


 * Serving Flask app '__main__'
 * Debug mode: off
Ngrok public URL: NgrokTunnel: "https://b0b2-35-203-133-124.ngrok-free.app" -> "http://localhost:5000"
Received fileUrl: http://res.cloudinary.com/dk06hi9th/image/upload/v1734811340/iwzeijtofauwgupulx74.png
Received ngrokUrl: https://cd17-154-208-41-213.ngrok-free.app
POST request successful: {'statusCode': 200, 'messsage': 'File data recieved successfully', 'success': True}
